<a href="https://colab.research.google.com/github/AllanRodrigo/Data_Science_Alura/blob/main/Notebooks/Semana_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalando Dependências para rodar no Colab**

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

# **Configuração do Spark IU**

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [5]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [12]:
get_ipython().system_raw('./ngrok authtoken 2Hy12AcUfWTBYgpyEw2f6N4N8eD_5SpEUjZRscsXFDfGo5UAH')
get_ipython().system_raw('./ngrok http 4050 &')

In [13]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://e9a7-35-229-204-95.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":2,"gauge":0,"rate1":0.0000013009854262630742,"rate5":0.000502767342969093,"rate15":0.0006138694125331011,"p50":30850307908.5,"p90":30884502193,"p95":30884502193,"p99":30884502193},"http":{"count":4,"rate1":0.0000015781750977357288,"rate5":0.0009098454089699078,"rate15":0.0011874887587908914,"p50":2645137.5,"p90":2842845,"p95":2842845,"p99":2842845}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://e9a7-35-229-204-95.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":5,"gauge":0,"rate1":0.000001196964374947491,"rate5":0.0004945637233281636,"rate15":0.0006815207073372453,"p50":30882910222,"p90":31221952168,"p95":31221952168,"p99":31221952168},"http":{"count":9,"ra

In [11]:
spark